In [125]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

In [126]:
stocks = pd.read_csv("../Project 1/sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [127]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
base_url = "https://sandbox.iexapis.com/stable/stock/"
col_names = ["Ticker", "Price", "Price-to-Earnings Ratio"]

In [128]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [129]:
symbol_groups = list(chunks(stocks["Ticker"], 100))

symbol_strings = []

final_dataframe = pd.DataFrame(columns = col_names)

for group in symbol_groups:
    symbol_strings.append(','.join(group))

for symbol_string in symbol_strings:
    batch_api_call_url = f"{base_url}market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == "WLTW":
            continue
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["quote"]["latestPrice"],
                    data[symbol]["quote"]["peRatio"]
                ],
                index = col_names
            ),
            ignore_index = True
        )

In [130]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio
0,A,140.46,17.62
1,AAL,19.26,-3.93
2,AAP,224.30,23.53
3,AAPL,177.32,29.4
4,ABBV,149.11,35.48
...,...,...,...
499,YUM,128.00,23.99
500,ZBH,126.56,31.25
501,ZBRA,449.80,14.42
502,ZION,75.84,7.25


In [131]:
final_dataframe.sort_values("Price-to-Earnings Ratio", inplace=True)
final_dataframe = final_dataframe[final_dataframe["Price-to-Earnings Ratio"] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [132]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio
0,F,18.730,2.06
1,BIO,644.350,2.3
2,SYF,45.750,3.16
3,EBAY,62.420,3.28
4,LEN,94.380,3.35
5,PHM,49.600,3.35
6,COO,403.640,3.45
7,HPE,17.530,3.5
8,DD,83.340,3.51
9,DOW,62.660,3.67


In [133]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Please enter your portfolio size")
    
    try:
        portfolio_size = int(portfolio_size)
    except ValueError:
        print("The value entered was not a number! Enter again :")
        portfolio_size = input()
        portfolio_size = int(portfolio_size)

In [134]:
portfolio_input()

Please enter your portfolio size10000000


In [135]:
position_size = portfolio_size/final_dataframe.shape[0]
position_size

200000.0

In [136]:
for row in final_dataframe.index:
    final_dataframe.loc[row, "Number of shares to buy"] = math.floor(position_size/final_dataframe.loc[row, "Price"])

In [137]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of shares to buy
0,F,18.730,2.06,10678.0
1,BIO,644.350,2.3,310.0
2,SYF,45.750,3.16,4371.0
3,EBAY,62.420,3.28,3204.0
4,LEN,94.380,3.35,2119.0
5,PHM,49.600,3.35,4032.0
6,COO,403.640,3.45,495.0
7,HPE,17.530,3.5,11409.0
8,DD,83.340,3.51,2399.0
9,DOW,62.660,3.67,3191.0


In [138]:
symbol = "AAPL"
batch_api_call_url = f"{base_url}market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(batch_api_call_url).json()
data

{'YUM': {'advanced-stats': {'beta': 0.6761676885386002,
   'totalCash': 1027019595,
   'currentDebt': 1356770329,
   'revenue': 6708040840,
   'grossProfit': 4904985875,
   'totalRevenue': 6606873914,
   'EBITDA': 2267225376,
   'revenuePerShare': 22.55,
   'revenuePerEmployee': 190763.79,
   'debtToEquity': -0.8352170932223473,
   'profitMargin': 0.25275437423340513,
   'enterpriseValue': 46443773336,
   'enterpriseValueToRevenue': 7.46,
   'priceToSales': 5.9,
   'priceToBook': -4.79,
   'forwardPERatio': 25.6313369160017,
   'pegRatio': 0.4726046651651539,
   'peHigh': 12.62286734642619,
   'peLow': 6.535843385072451,
   'week52highDate': '2021-12-26',
   'week52lowDate': '2021-02-25',
   'putCallRatio': 0.893038798389124,
   'companyName': 'Yum Brands Inc.',
   'marketcap': 37482468704,
   'week52high': 143.58,
   'week52low': 101.82,
   'week52highSplitAdjustOnly': 141.17,
   'week52highDateSplitAdjustOnly': '2021-12-23',
   'week52lowSplitAdjustOnly': 104.59,
   'week52lowDateSpl

In [139]:
rv_cols = ["Ticker",
           "Price",
           "Number of shares to buy",
           "Price-to-Earnings Ratio",
           "PE Percentile",
           "Price-to-Book Ratio",
           "PB Percentile",
           "Price-to-Sales Ratio",
           "PS Percentile",
           "EV/EBITDA",
           "EV/EBITDA Percentile",
           "EV/GP",
           "EV/GP Percentile",
           "RV Score"
          ]

In [140]:
rv_dataframe = pd.DataFrame(columns = rv_cols)
rv_dataframe

def try_division(val1, val2):
    try:
        x = val1/val2
        return x
    except TypeError:
        return np.NaN

In [141]:
for symbol_string in symbol_strings:
    batch_api_call_url = f"{base_url}market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == "WLTW":
            continue
        
        ev = data[symbol]["advanced-stats"]["enterpriseValue"]
        EBITDA = data[symbol]["advanced-stats"]["EBITDA"]
        gp = data[symbol]["advanced-stats"]["grossProfit"]
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["quote"]["latestPrice"],
                    'N/A',
                    data[symbol]["quote"]["peRatio"],
                    'N/A',
                    data[symbol]["advanced-stats"]["priceToBook"],
                    'N/A',
                    data[symbol]["advanced-stats"]["priceToSales"],
                    'N/A',
                    try_division(ev, EBITDA),
                    'N/A',
                    try_division(ev, gp),
                    'N/A',
                    'N/A'
                ],
                index = rv_cols
            ),
            ignore_index = True
        )

In [142]:
rv_dataframe

,Ticker,Price,Number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,141.26,N/A,17.29,N/A,3.84,N/A,3.37,N/A,12.465691,N/A,6.290456,N/A,N/A
1,AAL,19.40,N/A,-3.76,N/A,-1.67,N/A,0.5163,N/A,-7.491240,N/A,1.504951,N/A,N/A
2,AAP,225.80,N/A,22.87,N/A,4.35,N/A,1.31,N/A,12.782130,N/A,2.882481,N/A,N/A
3,AAPL,174.27,N/A,28.7,N/A,41.0,N/A,7.73,N/A,23.294605,N/A,18.096361,N/A,N/A
4,ABBV,151.42,N/A,35.48,N/A,18.91,N/A,4.73,N/A,11.994231,N/A,8.644275,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,124.00,N/A,24.02,N/A,-4.77,N/A,5.8,N/A,20.704780,N/A,9.835312,N/A,N/A
500,ZBH,125.57,N/A,32.14,N/A,2.07,N/A,3.39,N/A,18.305294,N/A,5.985035,N/A,N/A
501,ZBRA,437.19,N/A,13.96,N/A,4,N/A,2.09,N/A,11.191231,N/A,4.598742,N/A,N/A
502,ZION,74.66,N/A,7.18,N/A,1.59,N/A,3.39,N/A,5.728446,N/A,2.988316,N/A,N/A


In [143]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 
              'Price-to-Sales Ratio', 'EV/EBITDA',
              'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

In [144]:
metrics = {           
           "Price-to-Earnings Ratio" : "PE Percentile",
           "Price-to-Book Ratio" : "PB Percentile",
           "Price-to-Sales Ratio" : "PS Percentile",
           "EV/EBITDA" : "EV/EBITDA Percentile",
           "EV/GP" : "EV/GP Percentile",
}

In [145]:
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = (stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric]))/100

In [146]:
rv_dataframe

,Ticker,Price,Number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,141.26,N/A,17.29,0.405754,3.84,0.579365,3.3700,0.537698,12.465691,0.458333,6.290456,0.490079,N/A
1,AAL,19.40,N/A,-3.76,0.051587,-1.67,0.05754,0.5163,0.063492,-7.491240,0.019841,1.504951,0.06746,N/A
2,AAP,225.80,N/A,22.87,0.53869,4.35,0.668651,1.3100,0.246032,12.782130,0.472222,2.882481,0.172619,N/A
3,AAPL,174.27,N/A,28.70,0.666667,41.00,0.974206,7.7300,0.873016,23.294605,0.805556,18.096361,0.93254,N/A
4,ABBV,151.42,N/A,35.48,0.781746,18.91,0.936508,4.7300,0.734127,11.994231,0.436508,8.644275,0.690476,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,124.00,N/A,24.02,0.565476,-4.77,0.051587,5.8000,0.80754,20.704780,0.75,9.835312,0.744048,N/A
500,ZBH,125.57,N/A,32.14,0.734127,2.07,0.338294,3.3900,0.546627,18.305294,0.668651,5.985035,0.462302,N/A
501,ZBRA,437.19,N/A,13.96,0.315476,4.00,0.625,2.0900,0.390873,11.191231,0.39881,4.598742,0.331349,N/A
502,ZION,74.66,N/A,7.18,0.132937,1.59,0.232143,3.3900,0.546627,5.728446,0.107143,2.988316,0.180556,N/A


In [147]:
from statistics import mean

In [148]:
for row in rv_dataframe.index:
    percentiles = []
    for metric in metrics.keys():
        percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, "RV Score"] = mean(percentiles)

In [149]:
rv_dataframe.sort_values("RV Score", ascending=True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)

In [150]:
rv_dataframe

,Ticker,Price,Number of shares to buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,F,18.69,N/A,2.07,0.061508,0.7440,0.083333,0.2649,0.013889,1.739571,0.02381,0.934564,0.037698,0.044048
1,GM,50.15,N/A,3.80,0.085317,0.6230,0.06746,0.2971,0.017857,1.099285,0.021825,0.957072,0.039683,0.046429
2,AAL,19.40,N/A,-3.76,0.051587,-1.6700,0.05754,0.5163,0.063492,-7.491240,0.019841,1.504951,0.06746,0.051984
3,HPQ,37.81,N/A,3.68,0.079365,-12.6700,0.039683,0.3308,0.021825,4.074617,0.051587,1.764920,0.089286,0.056349
4,UNM,30.88,N/A,7.66,0.138889,0.5441,0.061508,0.4727,0.051587,3.704399,0.045635,0.442502,0.005952,0.060714
5,L,63.98,N/A,5.18,0.097222,0.4392,0.059524,0.5346,0.065476,3.717844,0.047619,0.967452,0.041667,0.062302
6,SYF,44.94,N/A,3.19,0.065476,0.9379,0.115079,0.7098,0.099206,2.049777,0.025794,0.734151,0.025794,0.06627
7,PHM,49.60,N/A,3.44,0.073413,0.8515,0.10119,0.4557,0.047619,2.427437,0.029762,1.695504,0.085317,0.06746
8,PRU,120.00,N/A,6.57,0.113095,0.7555,0.087302,0.6380,0.079365,4.798948,0.073413,0.641268,0.015873,0.07381
9,PFG,79.25,N/A,6.22,0.111111,0.6311,0.071429,0.7260,0.105159,4.310637,0.063492,0.713190,0.021825,0.074603


In [151]:
portfolio_input()

Please enter your portfolio size10000000


In [152]:
position_size = portfolio_size/rv_dataframe.shape[0]

In [153]:
for row in rv_dataframe.index:
    rv_dataframe.loc[row, "Number of shares to buy"] = math.floor(position_size/rv_dataframe.loc[row, "Price"])

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [154]:
writer = pd.ExcelWriter("value strategy.xlsx", engine="xlsxwriter")
rv_dataframe.to_excel(writer, "Value Strategy", index = False)

In [155]:
background_color = "0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

number_format = writer.book.add_format(
    {
        "num_format": "0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

float_format = writer.book.add_format(
    {
        "num_format": "0.0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

percent_format = writer.book.add_format(
    {
        "num_format": "0.0%",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

In [156]:
column_formats = {
         "A" : ["Ticker", string_format],
         "B" : ["Price", dollar_format],
         "C" : ["Number of shares to buy", number_format],
         "D" : ["Price-to-Earnings Ratio", float_format],
         "E" : ["PE Percentile", percent_format],
         "F" : ["Price-to-Book Ratio", float_format],
         "G" : ["PB Percentile", percent_format],
         "H" : ["Price-to-Sales Ratio", float_format],
         "I" : ["PS Percentile", percent_format],
         "J" : ["EV/EBITDA", float_format],
         "K" : ["EV/EBITDA Percentile", percent_format],
         "L" : ["EV/GP", float_format],
         "M" : ["EV/GP Percentile", percent_format],
         "N" : ["RV Score", percent_format]
}

In [157]:
for column in column_formats.keys():
    writer.sheets["Value Strategy"].set_column(f"{column}:{column}", 25, column_formats[column][1])
    writer.sheets["Value Strategy"].write(f"{column}1", column_formats[column][0], column_formats[column][1])

In [158]:
writer.save()